In [7]:
import os
from pathlib import Path
import fiona
import geopandas as gpd


In [8]:
gdb = "/import/beegfs/CMIP6/jdpaul3/hydroviz_data/gis/GeospatialFabric_National.gdb"
fiona.listlayers(gdb)

['POIs',
 'one',
 'nhdflowline_en',
 'nhdflowline',
 'regionOutletDA',
 'nhruNationalIdentifier',
 'nsegmentNationalIdentifier']

In [9]:
poi_gdf = gpd.read_file(gdb, layer='POIs', encoding='utf-8')
gf_seg_gdf = gpd.read_file(gdb, layer='nsegmentNationalIdentifier', encoding='utf-8')

In [46]:
# join POI to segments
gf_seg = gf_seg_gdf[['seg_id_nat', 'POI_ID']]
gf_seg['POI_ID'] = gf_seg['POI_ID'].astype(int)

/tmp/ipykernel_1406911/647745238.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gf_seg['POI_ID'] = gf_seg['POI_ID'].astype(int)


In [47]:
poi_gdf.columns

Index(['COMID', 'FDATE', 'RESOLUTION', 'GNIS_ID', 'GNIS_NAME', 'LENGTHKM',
       'REACHCODE', 'FLOWDIR', 'WBAREACOMI', 'FTYPE', 'FCODE', 'SHAPE_LENG',
       'ENABLED', 'HYDROSEQ', 'LEVELPATHI', 'DNLEVELPAT', 'DIVERGENCE',
       'STARTFLAG', 'SO', 'SC', 'TRAV_TIME', 'MAXELEVSMO', 'FROMMEAS',
       'TOMEAS', 'INC_DA', 'CUMDRAINAG', 'TOCOMID', 'TOCOMID2', 'FROMCOMID',
       'nseg_flag', 'Prod_Unit', 'Type_GagesII', 'Type_Ref', 'Type_RFC',
       'Type_SPRW', 'Type_User', 'Type_Region', 'Type_Conf', 'Type_WBOut',
       'Type_WBIn', 'Type_Con', 'Type_Travel', 'Type_Elev',
       'poi_gage_segment_orig', 'poi_gage_segment_comment', 'poi_gage_id',
       'poi_gage_segment', 'region', 'Shape_Le_1', 'headwaterArea',
       'geometry'],
      dtype='object')

In [48]:
poi = poi_gdf[['COMID', 'GNIS_NAME', 'poi_gage_id']]
poi['COMID'] = poi['COMID'].astype(int)
#poi['poi_gage_id'] = poi['poi_gage_id'].astype(int)

/tmp/ipykernel_1406911/4260413834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi['COMID'] = poi['COMID'].astype(int)


In [49]:
gf_seg = gf_seg.join(poi.set_index('COMID'), on='POI_ID')

In [51]:
gf_seg.where(gf_seg['poi_gage_id'] != "0").dropna()

,seg_id_nat,POI_ID,GNIS_NAME,poi_gage_id
14,15.0,3923.0,Mattawamkeag River,01030500
42,43.0,721640.0,Allagash River,01011000
47,48.0,724696.0,Fish River,01013500
67,68.0,805113.0,Aroostook River,01017000
79,80.0,805443.0,Aroostook River,01015800
...,...,...,...,...
56833,56834.0,26448034.0,,50047560
56835,56836.0,26448088.0,R¿o de la Plata,50043800
56843,56844.0,26448568.0,R¿o Grande De Loiza,50050900
56846,56847.0,26448598.0,R¿o Maunabo,50090500


In [44]:
gf_seg.where(gf_seg['poi_gage_id'] != 0).dropna().min()

seg_id_nat          15.0
POI_ID            3923.0
GNIS_NAME               
poi_gage_id    1011000.0
dtype: object

In [45]:
gf_seg.where(gf_seg['poi_gage_id'] != 0).dropna().max()

seg_id_nat          56853.0
POI_ID          914690002.0
GNIS_NAME      Zumbro River
poi_gage_id     810464660.0
dtype: object